In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

bevolking = CBS.bevolking().iloc[0]
#display(bevolking.BevolkingOpDeEersteVanDeMaand)
#display(bevolking['per 100k'])

def addstats(df):
  global dagoverzicht

  # de aanname is dat df al gegroepeerd is op datum, en dat de kolommen de dagtotalen zijn
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for stat in df.columns:
    # nieuw = dagtotaal, zet missende dagtotalen op 0
    dagoverzicht[f'{stat} (nieuw)'] = dagoverzicht[stat].fillna(0).astype(int)
    # kolom met eerder de dagtotalen = cumulatieve som over dagtotalen
    dagoverzicht[stat] = dagoverzicht[f'{stat} (nieuw)'].cumsum()
    # factor voor 100k
    dagoverzicht[f'{stat} per 100.000'] = dagoverzicht[stat] * bevolking['per 100k']
    # verschil met 7 dagen terug
    dagoverzicht[f'{stat} 7d'] = (dagoverzicht[stat] - dagoverzicht[stat].shift(6)).fillna(0).astype(int)
    # en weer factor 100 k
    dagoverzicht[f'{stat} 7d per 100.000'] = dagoverzicht[f'{stat} 7d'] * bevolking['per 100k']

In [2]:
@run('set up base frame + overleden + positief getest')
def cell():
  # 2 vliegen in 1 klap -- aantallen pos + overleden, en laatste datum met data voor de datum range
  df = RIVM.csv('COVID-19_aantallen_gemeente_per_dag').rename(columns={
    'Total_reported': 'Positief getest',
    'Deceased': 'Overleden',
    'Date_of_publication': 'Datum',
    'Date_of_report': 'Today',
  })
  # sloop tijd van de datum en zet om in datetime object
  df['Datum'] = pd.to_datetime(df.Datum.str.replace(' .*', '', regex=True))
  #df['Today'] = pd.to_datetime(df.Today.str.replace(' .*', '', regex=True))

  global dagoverzicht
  # maak leeg dataframe met een rij voor elke dag van 2020-02-27 tm Date_of_publication
  dagoverzicht = pd.DataFrame(index=pd.date_range(start='2020-02-27', end=df.Datum.max()))
  # noem de index Key
  dagoverzicht.index.name='Key'
  # vul de datum kolom
  dagoverzicht['Datum'] = dagoverzicht.index.strftime('%Y-%m-%d')
  # vaste waarde voor LandCode
  dagoverzicht['LandCode'] = 'NL'

  # sommeer pos en overl op datum en voeg toe aan dagoverzicht
  addstats(df.groupby(['Datum']).agg({'Positief getest': 'sum', 'Overleden': 'sum'}))
  for col in ['Overleden', 'Positief getest']:
    dagoverzicht[f'{col} week-1'] = dagoverzicht[f'{col} 7d'].shift(7).fillna(0)
  display(dagoverzicht.head(10))

set up base frame + overleden + positief getest


rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-13@14-15.csv exists
loading rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-13@14-15.csv.gz


,Datum,LandCode,Positief getest,Overleden,Positief getest (nieuw),Positief getest per 100.000,Positief getest 7d,Positief getest 7d per 100.000,Overleden (nieuw),Overleden per 100.000,Overleden 7d,Overleden 7d per 100.000,Overleden week-1,Positief getest week-1
Key,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,0,0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0.0,0.0
2020-02-28,2020-02-28,NL,1,0,1,0.005723,0,0.000000,0,0.000000,0,0.000000,0.0,0.0
2020-02-29,2020-02-29,NL,2,0,1,0.011445,0,0.000000,0,0.000000,0,0.000000,0.0,0.0
2020-03-01,2020-03-01,NL,5,0,3,0.028613,0,0.000000,0,0.000000,0,0.000000,0.0,0.0
2020-03-02,2020-03-02,NL,10,0,5,0.057226,0,0.000000,0,0.000000,0,0.000000,0.0,0.0
2020-03-03,2020-03-03,NL,14,0,4,0.080116,0,0.000000,0,0.000000,0,0.000000,0.0,0.0
2020-03-04,2020-03-04,NL,24,0,10,0.137341,24,0.137341,0,0.000000,0,0.000000,0.0,0.0
2020-03-05,2020-03-05,NL,41,0,17,0.234625,40,0.228902,0,0.000000,0,0.000000,0.0,0.0
2020-03-06,2020-03-06,NL,81,0,40,0.463527,79,0.452082,0,0.000000,0,0.000000,0.0,0.0


In [3]:
@run('ziekenhuisopnames')
def cell():
  df = RIVM.csv('COVID-19_ziekenhuisopnames').rename(columns={
    'Hospital_admission': 'Ziekenhuisopnames',
    'Date_of_statistics': 'Datum',
  })
  # datum naar datetime
  df['Datum'] = pd.to_datetime(df.Datum.str.replace(' .*', '', regex=True))
  # sommeer op datum en voeg toe aan dagoverzicht
  addstats(df.groupby(['Datum']).agg({'Ziekenhuisopnames': 'sum'}))
  dagoverzicht['Ziekenhuisopnames week-1'] = dagoverzicht['Ziekenhuisopnames 7d'].shift(7).fillna(0)
  display(dagoverzicht.head())

ziekenhuisopnames


rivm/COVID-19_ziekenhuisopnames-2021-02-13@14-15.csv exists
loading rivm/COVID-19_ziekenhuisopnames-2021-02-13@14-15.csv.gz


,Datum,LandCode,Positief getest,Overleden,Positief getest (nieuw),Positief getest per 100.000,Positief getest 7d,Positief getest 7d per 100.000,Overleden (nieuw),Overleden per 100.000,Overleden 7d,Overleden 7d per 100.000,Overleden week-1,Positief getest week-1,Ziekenhuisopnames,Ziekenhuisopnames (nieuw),Ziekenhuisopnames per 100.000,Ziekenhuisopnames 7d,Ziekenhuisopnames 7d per 100.000,Ziekenhuisopnames week-1
Key,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,0,0,0,0.000000,0,0.0,0,0.0,0,0.0,0.0,0.0,3,3,0.017168,0,0.0,0.0
2020-02-28,2020-02-28,NL,1,0,1,0.005723,0,0.0,0,0.0,0,0.0,0.0,0.0,6,3,0.034335,0,0.0,0.0
2020-02-29,2020-02-29,NL,2,0,1,0.011445,0,0.0,0,0.0,0,0.0,0.0,0.0,8,2,0.045780,0,0.0,0.0
2020-03-01,2020-03-01,NL,5,0,3,0.028613,0,0.0,0,0.0,0,0.0,0.0,0.0,17,9,0.097284,0,0.0,0.0
2020-03-02,2020-03-02,NL,10,0,5,0.057226,0,0.0,0,0.0,0,0.0,0.0,0.0,27,10,0.154509,0,0.0,0.0


In [4]:
@run('reproductiegetal en besmettelijkheid')
def cell():
  global dagoverzicht

  datasets = [
    ('COVID-19_reproductiegetal', 'Rt_avg', 'Reproductiegetal'),
    ('COVID-19_prevalentie', 'prev_avg', 'Besmettelijk'),
  ]
  for dataset, source, target in datasets:
    # laad de dataset
    df = RIVM.json(dataset).rename(columns={source: target})
    # Date naar datetime index voor de merge
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    # merge alleen de target kolom
    df = df[[target]]
    # voeg to aan dagoverzicht
    dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
    # vul lege waarden met 0
    dagoverzicht[target] = dagoverzicht[target].fillna(0)
  # per 100k factor
  dagoverzicht['Besmettelijk per 100.000'] = (dagoverzicht['Besmettelijk']  * bevolking['per 100k']).round(0)
  display(dagoverzicht)

reproductiegetal en besmettelijkheid


downloading rivm/COVID-19_reproductiegetal-2021-02-13@14-15.json


loading rivm/COVID-19_reproductiegetal-2021-02-13@14-15.json.gz


downloading rivm/COVID-19_prevalentie-2021-02-13@14-15.json


loading rivm/COVID-19_prevalentie-2021-02-13@14-15.json.gz


,Datum,LandCode,Positief getest,Overleden,Positief getest (nieuw),Positief getest per 100.000,Positief getest 7d,Positief getest 7d per 100.000,Overleden (nieuw),Overleden per 100.000,...,Positief getest week-1,Ziekenhuisopnames,Ziekenhuisopnames (nieuw),Ziekenhuisopnames per 100.000,Ziekenhuisopnames 7d,Ziekenhuisopnames 7d per 100.000,Ziekenhuisopnames week-1,Reproductiegetal,Besmettelijk,Besmettelijk per 100.000
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,0,0,0,0.000000,0,0.000000,0,0.000000,...,0.0,3,3,0.017168,0,0.000000,0.0,2.06,6501.0,37.0
2020-02-28,2020-02-28,NL,1,0,1,0.005723,0,0.000000,0,0.000000,...,0.0,6,3,0.034335,0,0.000000,0.0,2.10,8184.0,47.0
2020-02-29,2020-02-29,NL,2,0,1,0.011445,0,0.000000,0,0.000000,...,0.0,8,2,0.045780,0,0.000000,0.0,1.99,10266.0,59.0
2020-03-01,2020-03-01,NL,5,0,3,0.028613,0,0.000000,0,0.000000,...,0.0,17,9,0.097284,0,0.000000,0.0,1.83,12757.0,73.0
2020-03-02,2020-03-02,NL,10,0,5,0.057226,0,0.000000,0,0.000000,...,0.0,27,10,0.154509,0,0.000000,0.0,1.79,15691.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-09,2021-02-09,NL,1009588,14508,1765,5777.429108,20606,117.919096,85,83.022918,...,23792.0,43181,180,247.105915,970,5.550884,1105.0,0.00,0.0,0.0
2021-02-10,2021-02-10,NL,1012793,14586,3205,5795.769917,19595,112.133587,78,83.469278,...,23117.0,43355,174,248.101640,962,5.505104,1064.0,0.00,0.0,0.0
2021-02-11,2021-02-11,NL,1017237,14660,4444,5821.200979,19706,112.768791,74,83.892747,...,22924.0,43488,133,248.862741,917,5.247589,1027.0,0.00,0.0,0.0


In [5]:
@run('uitgevoerde testen')
def cell():
  df = RIVM.csv('COVID-19_uitgevoerde_testen').rename(columns={
    'Date_of_statistics': 'Datum',
    'Tested_with_result': 'GGD getest',
    'Tested_positive': 'GGD getest positief',
  })
  df['Datum'] = pd.to_datetime(df.Datum)
  df = df.groupby(['Datum']).agg({'GGD getest': 'sum', 'GGD getest positief': 'sum'})
  display(df)

  global dagoverzicht
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for col in df.columns:
    dagoverzicht[col] = dagoverzicht[col].fillna(0)

  dagoverzicht['GGD percentage positief'] = (dagoverzicht['GGD getest positief'] / dagoverzicht['GGD getest']).fillna(0)

  dagoverzicht['GGD getest (7 daags)'] = dagoverzicht['GGD getest'].rolling(7).sum().fillna(0)
  dagoverzicht['GGD getest positief (7 daags)'] = dagoverzicht['GGD getest positief'].rolling(7).sum().fillna(0)

  dagoverzicht['GGD percentage positief (7 daags)'] = (dagoverzicht['GGD getest positief (7 daags)'] / dagoverzicht['GGD getest (7 daags)']).fillna(0)

  dagoverzicht['GGD getest (cumulatief)'] = dagoverzicht['GGD getest'].cumsum()
  dagoverzicht['GGD getest positief (cumulatief)'] = dagoverzicht['GGD getest positief'].cumsum()

  dagoverzicht['GGD percentage positief (cumulatief)'] = (dagoverzicht['GGD getest positief (cumulatief)'] / dagoverzicht['GGD getest (cumulatief)']).fillna(0)

  display(dagoverzicht.head())

uitgevoerde testen


downloading rivm/COVID-19_uitgevoerde_testen-2021-02-13@14-15.csv


loading rivm/COVID-19_uitgevoerde_testen-2021-02-13@14-15.csv.gz


,GGD getest,GGD getest positief
Datum,,
2020-06-01,1552,73
2020-06-02,6819,203
2020-06-03,8867,165
2020-06-04,9339,171
2020-06-05,9464,135
...,...,...
2021-02-07,94,6
2021-02-08,24488,3010
2021-02-09,30912,3588


,Datum,LandCode,Positief getest,Overleden,Positief getest (nieuw),Positief getest per 100.000,Positief getest 7d,Positief getest 7d per 100.000,Overleden (nieuw),Overleden per 100.000,...,Besmettelijk per 100.000,GGD getest,GGD getest positief,GGD percentage positief,GGD getest (7 daags),GGD getest positief (7 daags),GGD percentage positief (7 daags),GGD getest (cumulatief),GGD getest positief (cumulatief),GGD percentage positief (cumulatief)
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,0,0,0,0.000000,0,0.0,0,0.0,...,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,2020-02-28,NL,1,0,1,0.005723,0,0.0,0,0.0,...,47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-29,2020-02-29,NL,2,0,1,0.011445,0,0.0,0,0.0,...,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-01,2020-03-01,NL,5,0,3,0.028613,0,0.0,0,0.0,...,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-02,2020-03-02,NL,10,0,5,0.057226,0,0.0,0,0.0,...,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
@run('LCPS')
def cell():
  # laad dataset
  df = LCPS.csv('covid-19').rename(columns={
    'IC_Bedden_COVID': 'LCPS IC Bedden COVID',
    'IC_Bedden_Non_COVID': 'LCPS IC Bedden Non COVID',
    'Kliniek_Bedden': 'LCPS Kliniek Bedden COVID',
    'IC_Nieuwe_Opnames_COVID': 'LCPS IC Nieuwe Opnames COVID',
    'Kliniek_Nieuwe_Opnames_COVID': 'LCPS Kliniek Nieuwe Opnames COVID',
  })
  # datum naar datetime index voor merge
  df['Datum'] = pd.to_datetime(df['Datum'], format='%d-%m-%Y')
  df.set_index('Datum', inplace=True)

  # sommeer op datum
  df = df.groupby(['Datum']).agg({col: 'sum' for col in df.columns})

  # toename = waarde - vorige
  df['LCPS IC Bedden COVID (toename)'] = (df['LCPS IC Bedden COVID'] - df['LCPS IC Bedden COVID'].shift(1)).fillna(0)
  df['LCPS Kliniek Bedden COVID (toename)'] = (df['LCPS Kliniek Bedden COVID'] - df['LCPS Kliniek Bedden COVID'].shift(1)).fillna(0)

  global dagoverzicht
  # voeg toe aan dagoverzicht
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for col in df.columns:
    # vervang lege waarden door 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0).astype(int)

LCPS


lcps/covid-19-2021-02-13@13-00.csv exists
loading lcps/covid-19-2021-02-13@13-00.csv.gz


In [7]:
@run('corrections')
def cell():
  # laad corrections van mzelst
  df = GitHub.csv('mzelst/covid-19/contents/corrections/corrections_perday.csv')
  # date naar datetime index voor merge
  df['date'] = pd.to_datetime(df.date)
  df.set_index('date', inplace=True)
  columns =  {
    'net.infection': 'Positief getest (toename)',
    'net.hospitals': 'Ziekenhuisopnames (toename)',
    'net.deaths': 'Overleden (toename)'
  }
  # rename kolommen naar onze namen
  df = df.rename(columns=columns)[list(columns.values())]

  global dagoverzicht
  # voeg toe aan dagoverxicht
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for col in columns.values():
    # set lege waarden op 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0).astype(int)

corrections
https://api.github.com/repos/mzelst/covid-19/contents/corrections/corrections_perday.csv


downloading github/corrections_perday-2021-02-13@14-21.csv


In [8]:
@run('ESRI -> NICE')
def cell():
  global dagoverzicht
  df = ArcGIS.nice('f27f743476a142538e8054f7a7ce12e1')

  df['date'] = pd.to_datetime(df.date.str.replace(' .*', '', regex=True))
  df.set_index('date', inplace=True)

  # the NICE data as prepared by ESRI can just be used as-is, so just rename the columns
  for prefix, kind in [ ('ic', 'IC'), ('zkh', 'Ziekenhuis') ]:
    df = df.rename(columns={
      f'{prefix}NewIntake': f'NICE {kind} Bedden (intake)',
      f'{prefix}IntakeCount': f'NICE {kind} Bedden',
      f'{prefix}IntakeCumulative': f'NICE {kind} Bedden (cumulatief)',
      f'{prefix}DiedCumulative': f'NICE {kind} Overleden',
    })

  # remove the columns we don't use
  df = df[[col for col in df.columns if 'NICE' in col]]
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)

  # after the merge, dates missing in the ESRI/NICE set will be `nan`, so fill these
  for col in df.columns:
    # for cumulatief columns, fill-forward from last known value
    if 'cumulatief' in col:
      dagoverzicht[col] = dagoverzicht[col].ffill()
    # for non-cumulative and whatever remains in cumulative after fill forward (which will be leading nan's), fill with 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0)

  for kind in [ 'IC', 'Ziekenhuis' ]:
    dagoverzicht[f'NICE {kind} Bedden (toename)'] = (dagoverzicht[f'NICE {kind} Bedden'] - dagoverzicht[f'NICE {kind} Bedden'].shift(1)).fillna(0)

  for window, shift, past in [(3, 1, 'afgelopen '), (7, 0, '')]:
    dagoverzicht[f'NICE IC Bedden (intake) {past}{window}d'] = dagoverzicht['NICE IC Bedden (intake)'].shift(shift).rolling(window).sum().fillna(0)

  dagoverzicht['NICE IC Bedden (intake) week-1'] = dagoverzicht['NICE IC Bedden (intake) 7d'].shift(7).fillna(0)

ESRI -> NICE


downloading nice/f27f743476a142538e8054f7a7ce12e1_0-2021-02-13@13-26.csv


loading nice/f27f743476a142538e8054f7a7ce12e1_0-2021-02-13@13-26.csv.gz


In [9]:
@run('Personen')
def cell():
  global dagoverzicht
  dagoverzicht['Personen'] = bevolking.BevolkingOpDeEersteVanDeMaand
  display(dagoverzicht)

Personen


,Datum,LandCode,Positief getest,Overleden,Positief getest (nieuw),Positief getest per 100.000,Positief getest 7d,Positief getest 7d per 100.000,Overleden (nieuw),Overleden per 100.000,...,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,NICE IC Bedden (toename),NICE Ziekenhuis Bedden (toename),NICE IC Bedden (intake) afgelopen 3d,NICE IC Bedden (intake) 7d,NICE IC Bedden (intake) week-1,Personen
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,0,0,0,0.000000,0,0.000000,0,0.000000,...,2,67,64,0,0.0,0.0,0.0,0.0,0.0,17474693
2020-02-28,2020-02-28,NL,1,0,1,0.005723,0,0.000000,0,0.000000,...,3,70,67,0,1.0,3.0,0.0,0.0,0.0,17474693
2020-02-29,2020-02-29,NL,2,0,1,0.011445,0,0.000000,0,0.000000,...,1,71,67,0,1.0,0.0,0.0,0.0,0.0,17474693
2020-03-01,2020-03-01,NL,5,0,3,0.028613,0,0.000000,0,0.000000,...,7,79,75,0,2.0,8.0,1.0,0.0,0.0,17474693
2020-03-02,2020-03-02,NL,10,0,5,0.057226,0,0.000000,0,0.000000,...,10,90,86,0,-1.0,11.0,2.0,0.0,0.0,17474693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-09,2021-02-09,NL,1009588,14508,1765,5777.429108,20606,117.919096,85,83.022918,...,178,38459,1630,5573,-1.0,-21.0,72.0,161.0,221.0,17474693
2021-02-10,2021-02-10,NL,1012793,14586,3205,5795.769917,19595,112.133587,78,83.469278,...,170,38617,1623,5595,0.0,-7.0,70.0,161.0,221.0,17474693
2021-02-11,2021-02-11,NL,1017237,14660,4444,5821.200979,19706,112.768791,74,83.892747,...,146,38759,1601,5619,-3.0,-22.0,78.0,165.0,203.0,17474693


In [10]:
async def publish():
  global dagoverzicht

  m = (dagoverzicht == np.inf)
  df = dagoverzicht.loc[m.any(axis=1), m.any(axis=0)]
  display(df.head())
  m = (dagoverzicht == np.nan)
  df = dagoverzicht.loc[m.any(axis=1), m.any(axis=0)]
  display(df.head())

  os.makedirs('artifacts', exist_ok = True)
  dagoverzicht.to_csv('artifacts/DagOverzicht.csv', index=True)

  if knack:
    print('updating knack')
    df = dagoverzicht.assign(Key=dagoverzicht.index.strftime('%Y-%m-%d'))
    await knack.update(objectName='Dagoverzicht', df=df)
    await knack.update(objectName='LaatsteUpdate', df=pd.DataFrame([{'Key': 1, **{ f'Timestamp Dagoverzicht {provider.upper()}': ts for provider, ts in Cache.timestamps.items() if provider != 'github' }}]))
await publish()

""
Key


""
Key


updating knack


  0%|          | 0/348 [00:00<?, ?it/s]

  0%|          | 1/348 [00:00<04:05,  1.41it/s]

...


rate limit: 7 
API calls: create: 1, read: 2, update: 347, delete: 0, backoff: 1
errors:
  503: Service Unavailable: 2
  504: Gateway Time-out: 1


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.46it/s]

100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


rate limit: 7 
API calls: create: 0, read: 1, update: 1, delete: 0, backoff: 0
